В качестве генератора случайных чисел выберем когруэнтный метод из прошлой лаб раб 

In [1]:
import math
def congruent_method_generator():
    m =10**10
    x=get_x0(m)
    a=197
    while True:
        x = a*x%m
        yield x * 1.0 / m


def get_x0(m):
    for i in range(m-1,1,-1):
        if math.gcd(i,m) == 1:
            return i
    return 1

In [2]:
class RandomEventSimulator:

    def __init__(self):
        self._generator = congruent_method_generator()
    
    def check_event(self,p):
        _p = next(self._generator)
        return _p < p
    

In [3]:
p_list = [0.5, 0.1, 0.8]
N_SAMPLES = 10**6
simulator = RandomEventSimulator()


for p in p_list:
    answers = list()
    for i in range(N_SAMPLES):
        answers.append(simulator.check_event(p))
    happend_event_count = sum(answers)
    print(f"p: {p} happend events count :{happend_event_count}")


p: 0.5 happend events count :500247
p: 0.1 happend events count :100224
p: 0.8 happend events count :800504


In [6]:
class IndependentEventsSimulator:
    
    def __init__(self) -> None:
        self._random_event_simulator = RandomEventSimulator()
    
    def check_events(self,k):
        result_vector = list()

        for _k in k:
            result_vector.append(self._random_event_simulator.check_event(_k))
        
        return result_vector

In [7]:
N_SAMPLES = 10**6

k_list=[
    [0.1,0.2,0.3,0.4],
    [0.8,0.1,0.05,0.05],
    [0.5,0.5]
]

simulator = IndependentEventsSimulator()

for k in k_list:
    answers = list()
    for i in range(N_SAMPLES):
        answers.append(simulator.check_events(k))
    event_freq = [0 for i in k]
    for answer in answers:
        for i in range(len(k)):
            event_freq[i]+= answer[i]
    print(f"event freq:{event_freq}")


event freq:[99890, 200178, 300416, 399611]
event freq:[799972, 99647, 49879, 49916]
event freq:[500012, 500476]


In [16]:
import numpy as np
class NotSimpleEventSimulator():

    def __init__(self):
        self._generator = congruent_method_generator()
    
    def _first(self,it,condition):
        return next(x for x in it if condition(x))
    def check_event(self,k):
        p = next(self._generator)
        _cumsum = enumerate(np.cumsum(k))
        success_event = self._first(_cumsum, lambda x : p <=x[1])[0]
        bool_vector = [False for _ in k]
        bool_vector[success_event] = True
        return bool_vector

N_SAMPLES = 10**6

k_list=[
    [0.1,0.2,0.3,0.4],
    [0.8,0.1,0.05,0.05],
    [0.5,0.5]
]

simulator = NotSimpleEventSimulator()

for k in k_list:
    answers = list()
    for i in range(N_SAMPLES):
        answers.append(simulator.check_event(k))
    event_freq = [0 for i in k]
    for answer in answers:
        for i in range(len(k)):
            event_freq[i]+= answer[i]
    print(f"event freq:{event_freq}")



event freq:[100792, 200230, 298794, 400184]
event freq:[799771, 100168, 50018, 50043]
event freq:[499764, 500236]


In [11]:
class DependentEventGenerator:

    def __init__(self):
        self.generator = congruent_method_generator()
    
    def A_B(self,x1,x2,p_a,p_b_a):
        return x1<= p_a and x2<=p_b_a
    def A_not_B(self,x1,x2,p_a,p_b_a):
        return x1<= p_a and x2 > p_b_a
    def not_A_B(self,x1,x2,p_a,p_b_a):
        return x1> p_a and x2<= p_b_a
    def not_A_not_B(self,x1,x2,p_a,p_b_a):
        return x1 > p_a and x2> p_b_a
    
    def check_event(self,p_a,p_b_a):
        x1= next(self.generator)
        x2 = next(self.generator)
        p_b_not_a = 1 - p_b_a

        a_b = self.A_B(x1,x2,p_a,p_b_a)
        a_not_b = self.A_not_B(x1,x2,p_a,p_b_a)
        not_a_b = self.not_A_B(x1,x2,p_a,p_b_not_a)
        not_a_not_b = self.not_A_not_B(x1,x2,p_a,p_b_not_a)

        return [a_b,a_not_b,not_a_b,not_a_not_b]

def find_p_b(p_b_a,p_a):
    return p_b_a* p_a +(1-p_b_a)*(1-p_a)

def find_p_not_b_a(p_b_a,p_a):
    p_b = find_p_b(p_b_a,p_a)
    print(f"p_a:{p_a}, p_b_a: {p_b_a}, p_b:{p_b}")
    p_a_not_b = (p_a-1+p_b)/(2*p_b -1)
    return p_a_not_b*(1-p_b)/p_a

N_SAMPLES = 10**6
p_a= [0.1,0.4,0.3]
p_b_a =[0.2,0.1,0.4]
k = len(p_a)

p = p_a[0]
b = p_b_a[0]

p_b = p*b+(1-b)*(1-)

x = (1-(b*p+(1-b)*(1-p))-(1-p))/(1-2*p)

print(x)

p_ab = [p_a[i]*p_b_a[i] for i in range(k)]
p_a_not_b = [p_a[i]*(1-p_b_a[i]) for i in range(k)]
p_not_a_b =[(1- p_a[i])*find_p_not_b_a(p_b_a[i],p_a[i]) for i in range(k)]
p_not_a_not_b =[(1-p_a[i])*(1-p_not_a_b[i]) for i in range(k)]

simulator = DependentEventGenerator()

for i in range(k):
    answers = list()
    for j in range(N_SAMPLES):
        answers.append(simulator.check_event(p_a[i],p_b_a[i]))
    
    freq = [0 for _ in answers[0]]
    for answer in answers:
        for _i in range(len(answer)):
            freq[_i]+=answer[_i]
    
    print([p_ab[i],p_a_not_b[i],p_not_a_b[i],p_not_a_not_b[i]])
    print(freq)






-0.8000000000000002
p_a:0.1, p_b_a: 0.2, p_b:0.7400000000000001
p_a:0.4, p_b_a: 0.1, p_b:0.5800000000000001
p_a:0.3, p_b_a: 0.4, p_b:0.54
[0.020000000000000004, 0.08000000000000002, -0.779999999999999, 1.601999999999999]
[20528, 79982, 718159, 181331]
[0.04000000000000001, 0.36000000000000004, -0.07874999999999953, 0.6472499999999997]
[39986, 360153, 539884, 59977]
[0.12, 0.18, -2.1466666666666634, 2.2026666666666643]
[120581, 180081, 420312, 279026]


In [26]:
import itertools

keys = range(10)

print(list(itertools.combinations(keys,2)))

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 6), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9), (7, 8), (7, 9), (8, 9)]


In [31]:
from collections import namedtuple 
import copy


class FootballCupSimulator:
    def __init__(self,names,powers) -> None:
        self._commands_dict = dict()
        self._teams_count = len(names)
        self._team_type = namedtuple('Team',['name','power'])  
        self._match_simulator = RandomEventSimulator()
        for i in range(len(names)):
            self._commands_dict[names[i]] = self._team_type(names[i],powers[i])
    
    def play(self):
        simulator = NotSimpleEventSimulator()
        current_teams_count = self._teams_count
        commands_list = list(self._commands_dict.keys())
        round_count = 1
        while current_teams_count != 1:
            print(f"Commands in raund {round_count} : {commands_list}")
            pairs = list()
            for i in range(current_teams_count//2):
                frst_command = commands_list[0]
                commands_list.remove(frst_command)
                choose = simulator.check_event([1.0/len(commands_list) for _ in commands_list])
                snd_command = commands_list[choose.index(True)]
                commands_list.remove(snd_command)
                pairs.append([frst_command,snd_command])
            print(f"Pairs {pairs}")
            commands_list = self._play_round(pairs)
            current_teams_count //=2
            round_count +=1
        print(f"WINNER : {commands_list[0]}")
    
    def _play_round(self,pairs):
        winners = list()
        for pair in pairs:
            frst_team = self._commands_dict[pair[0]]
            snd_team = self._commands_dict[pair[1]]
            frst_win_p = (frst_team.power/(frst_team.power + snd_team.power))
            if self._match_simulator.check_event(frst_win_p):
                winners.append(pair[0])
            else:
                winners.append(pair[1])
        return winners



simulator = FootballCupSimulator(range(32),[i % 10 + 1 for i in  range(32)])
simulator.play()




Commands in raund 1 : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Pairs [[0, 31], [1, 30], [2, 29], [3, 25], [4, 12], [5, 24], [6, 28], [7, 19], [8, 13], [9, 16], [10, 26], [11, 20], [14, 27], [15, 18], [17, 22], [21, 23]]
Commands in raund 2 : [31, 30, 29, 25, 4, 24, 28, 19, 8, 9, 26, 11, 27, 15, 17, 21]
Pairs [[31, 19], [30, 21], [29, 27], [25, 8], [4, 15], [24, 9], [28, 17], [26, 11]]
Commands in raund 3 : [19, 21, 27, 8, 15, 9, 17, 11]
Pairs [[19, 11], [21, 9], [27, 8], [15, 17]]
Commands in raund 4 : [11, 9, 27, 17]
Pairs [[11, 27], [9, 17]]
Commands in raund 5 : [27, 9]
Pairs [[27, 9]]
WINNER : 27
